<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/ai_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PEFT Tuning LLama**

https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/llama

https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/llama/llama3-1-8

https://cloud.google.com/vertex-ai/generative-ai/docs/models/open-model-tuning

https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-use-supervised-tuning

**Project Structure**

Create a directory structure like this and upload the files in the bucket:

```
vertex-peft-pipeline/
├── pipeline.py
├── requirements.txt
└── src/
    ├── train.py
    └── Dockerfile
```

**Preparations**

Set your project ID and number as variables
```
export PROJECT_ID=$(gcloud config get-value project)
export PROJECT_NUMBER=$(gcloud projects describe ${PROJECT_ID} --format="value(projectNumber)")
```

Grant the Artifact Registry Writer role to the Cloud Build service account
```
gcloud projects add-iam-policy-binding ${PROJECT_ID} \
    --member="serviceAccount:${PROJECT_NUMBER}@cloudbuild.gserviceaccount.com" \
    --role="roles/artifactregistry.writer"
```

Make sure to create an Artifact Registry Repository
```
gcloud artifacts repositories create peft-tuning-repo \ --repository-format=docker \ --location=us-central1 \ --description="Docker repository for PEFT training images"
```


**Create a 'requirements.txt'**

```
google-cloud-aiplatform
kfp
```

**Create pipeline.py**

```
import os
from google.cloud import aiplatform

# --- Configuration ---
PROJECT_ID = "YOUR-PROJECT-ID"
REGION = "us-central1"
# We still need a staging bucket for the CustomJob
STAGING_BUCKET = "gs://deltorobarba-us-central1/pipeline-root"
DOCKER_REPO_NAME = "peft-tuning-repo"

# This token should be managed securely, e.g., via Vertex AI Secrets
HF_TOKEN = "YOUR-HUGGINGFACE-TOKEN"                     # <--- YOU NEED TO UPDATE THAT

# --- Docker Image Configuration ---
IMAGE_NAME = "peft-lora-trainer"
IMAGE_TAG = "latest"
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{DOCKER_REPO_NAME}/{IMAGE_NAME}:{IMAGE_TAG}"

# --- Main execution block ---
if __name__ == "__main__":
    aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

    # --- Define the arguments for the training script ---
    # These are the parameters that were previously in the pipeline definition
    MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
    DATASET_GCS_DIRECTORY = "gs://deltorobarba-us-central1/data"
    EPOCHS = 2
    # Define a unique output path for this job's artifacts
    ADAPTER_GCS_URI = f"{STAGING_BUCKET}/peft-output/{aiplatform.utils.timestamped_unique_name()}"


    # --- Create and run the CustomJob directly ---
    job = aiplatform.CustomJob(
        display_name="peft-lora-tuning-job",
        worker_pool_specs=[
            {
                "machine_spec": {
                    "machine_type": "g2-standard-4",
                    "accelerator_type": "NVIDIA_L4",
                    "accelerator_count": 1,
                },
                "replica_count": 1,
                "container_spec": {
                    "image_uri": IMAGE_URI,
                    "args": [
                        "--model-id", MODEL_ID,
                        "--dataset-gcs-directory", DATASET_GCS_DIRECTORY,
                        "--epochs", str(EPOCHS),
                        "--adapter-gcs-uri", ADAPTER_GCS_URI,
                    ],
                    "env": [{"name": "HF_TOKEN", "value": HF_TOKEN}],
                },
            }
        ],
    )

    job.run()

    print("--- Custom job submitted. ---")
    print(f"View the job here: {job.resource_name}")
    print(f"Adapter will be saved to: {ADAPTER_GCS_URI}")
```

**Create Dockerfile**

```
# In your Dockerfile

# Use a PyTorch base image with CUDA support
FROM pytorch/pytorch:2.1.0-cuda11.8-cudnn8-runtime

# Set the working directory
WORKDIR /app

# Install dependencies
RUN pip install --no-cache-dir --upgrade pip
# CORRECTED: Update libraries to their latest versions for Llama 3.1 support
RUN pip install --no-cache-dir \
    "transformers>=4.42.0" \
    "peft>=0.11.0" \
    "accelerate>=0.31.0" \
    "bitsandbytes>=0.43.0" \
    "datasets" \
    "torch" \
    "google-cloud-storage" \
    "google-cloud-aiplatform" \
    "gcsfs"

# Copy the training script into the container
COPY train.py .
# Set the entrypoint for the container
ENTRYPOINT ["python", "train.py"]
```

**Create train.py**

```
import argparse
import os

import torch
from datasets import load_dataset
from google.cloud import storage
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

def train_and_upload(args):
    """Loads model and dataset, performs PEFT training, and uploads to GCS."""

    # --- 1. Load Tokenizer and Model ---
    print("--- 1. Loading tokenizer and model ---")
    
    # Hugging Face token to access Llama 2
    # In a production pipeline, use Vertex AI Secrets for this
    hf_token = os.environ.get("HF_TOKEN")
    if not hf_token:
        raise ValueError("Hugging Face token not found in environment variable HF_TOKEN")

    # Configuration for loading the model in 4-bit precision
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(args.model_id, token=hf_token)
    # Llama 2 does not have a pad token, so we set it to the end-of-sentence token
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        args.model_id,
        quantization_config=bnb_config,
        device_map="auto", # Automatically place layers on available devices
        token=hf_token,
    )

    # Prepare model for k-bit training
    model = prepare_model_for_kbit_training(model)

    # --- 2. Configure PEFT LoRA ---
    print("--- 2. Configuring PEFT LoRA ---")
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        # Llama 3 has more projection layers (q, k, v, o)
        # CHANGE THIS LINE:
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
    
    # Add LoRA adapter to the model
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # --- 3. Load and Preprocess Dataset ---
    print(f"--- 3. Loading and processing dataset from: {args.dataset_gcs_directory} ---")
    
    # Construct the full path to the training file in the GCS bucket
    train_file = f"{args.dataset_gcs_directory}/train.jsonl"
    eval_file = f"{args.dataset_gcs_directory}/val.jsonl"

    
    # Load the dataset from the specified GCS path
    # We specify the format is 'json' and provide the file path
    dataset = load_dataset("json", data_files={"train": train_file}, split="train")

    def tokenize_function(examples):
        # The 'messages' field from your data is already in the correct format.
        # We can pass it directly to the tokenizer.
        # Note: We are accessing examples['messages'] which exists in your data.
        messages = examples['messages']
    
        tokenized_text = tokenizer.apply_chat_template(
            messages,
            truncation=True,
            padding="max_length",
            # Increase max_length as your new data format is much longer
            max_length=512,
            add_generation_prompt=False
        )
        return {"input_ids": tokenized_text}

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)


    # --- 4. Set Up Trainer ---
    print("--- 4. Setting up Trainer ---")
    
    # Output directory for the training artifacts within the container
    local_output_dir = "/tmp/output"

    training_args = TrainingArguments(
        output_dir=local_output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=args.epochs,
        logging_steps=10,
        fp16=True, # Use mixed precision
        save_total_limit=2,
        report_to="none", # Disable reporting to wandb/tensorboard for this example
        evaluation_strategy="steps", # Evaluate at regular intervals
        eval_steps=50, # How often to evaluate
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    # --- 5. Start Training ---
    print("--- 5. Starting training ---")
    trainer.train()
    print("--- Training finished ---")


    # --- 6. Save and Upload Adapter ---
    print(f"--- 6. Saving adapter to {local_output_dir} and uploading to {args.adapter_gcs_uri} ---")
    # Save the PEFT adapter weights
    trainer.save_model(local_output_dir)

    # Upload the contents of the local output directory to GCS
    storage_client = storage.Client()
    bucket_name = args.adapter_gcs_uri.replace("gs://", "").split("/")[0]
    destination_prefix = "/".join(args.adapter_gcs_uri.replace("gs://", "").split("/")[1:])
    bucket = storage_client.bucket(bucket_name)

    for root, _, files in os.walk(local_output_dir):
        for filename in files:
            local_path = os.path.join(root, filename)
            gcs_path = os.path.join(destination_prefix, os.path.relpath(local_path, local_output_dir))
            blob = bucket.blob(gcs_path)
            blob.upload_from_filename(local_path)
            print(f"Uploaded {local_path} to gs://{bucket_name}/{gcs_path}")

    print("--- Artifacts uploaded successfully ---")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-id", type=str, required=True, help="Base model ID from Hugging Face Hub")
    # Change dataset-id to dataset-gcs-directory to match the pipeline
    parser.add_argument("--dataset-gcs-directory", type=str, required=True, help="GCS directory containing train.jsonl")
    parser.add_argument("--epochs", type=int, default=1, help="Number of training epochs")
    parser.add_argument("--adapter-gcs-uri", type=str, required=True, help="GCS URI to save the trained adapter")

    args = parser.parse_args()
    train_and_upload(args)
```

**Load files, build dockerfile and start tuning job**

Open shell terminal in Google Cloud. Then copy the files from a GCS bucket into shell (the "." at the end means "copy it to my current directory"):

```
export GCS_SOURCE_PATH="gs://deltorobarba/tuning/vertex-peft-pipeline"
gcloud storage cp -r ${GCS_SOURCE_PATH} .
```

Navigate into the directory you just copied
```
cd vertex-peft-pipeline
```

Now run your build and pipeline submission commands
```
pip install -r requirements.txt
```

Replace with your actual project ID and desired region
```
export PROJECT_ID = "YOUR-PROJECT-ID"
export REGION="us-central1"
export DOCKER_REPO_NAME="peft-tuning-repo"
export IMAGE_NAME="peft-lora-trainer"
export IMAGE_URI="${REGION}-docker.pkg.dev/${PROJECT_ID}/${DOCKER_REPO_NAME}/${IMAGE_NAME}:latest"
```

Verify the Full Image Tag
```
echo ${IMAGE_URI}
```

Run the Correct Build Command, takes 5-10 min (make sure IAM permission are provided to compute and repository:
```
gcloud builds submit ./src --tag="${IMAGE_URI}"
```

Now run pipeline
```
python pipeline.py
```


**Understanding the Output Files**

The files you see in your GCS bucket are the components of your LoRA adapter. You are not saving a whole new 8-billion-parameter model, but just the small, efficient adapter "layers" that you trained. Here are the most important files:

* `adapter_model.safetensors`: This is the core of your output. It contains the actual trained weights of your LoRA adapter. The .safetensors format is a secure and fast way to store model weights.

* `adapter_config.json`: This is the configuration file for your adapter. It tells the PEFT library how the adapter was built (e.g., the LoRA rank (r), target_modules, etc.), so it knows how to correctly load the weights from adapter_model.safetensors and apply them to the base model.

* `tokenizer.json`, `tokenizer.model`, etc.: These files are a complete copy of the tokenizer from the base Llama 3.1 model. The trainer saves these to ensure that you use the exact same tokenizer for inference that you used during training, preventing any mismatch issues.

Fine Tuning of OpenAI GPT OSS on Google Cloud Vertex AI Colab Enterprise using NVIDIA A100

  https://lnkd.in/eM-jasdb  

Check Unsloth github and star it if you like to Fine Tune and Serve LLMs with a reduced memory footprint :  https://lnkd.in/eGgQaTQm



Also explore Unsloth AI fantastic resources around Google DeepMind Gemma 3 and  Gemma 3n resources for smaller models that better fit edge and Web AI use cases

Fine tuning Gemma 3 1B here : https://lnkd.in/e96kDCfH